In [1]:
import numpy
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show, output_notebook
from bokeh.embed import components
from bokeh.layouts import column
import pandas as pd
import numpy as np
from math import pi

from bokeh import events
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.plotting import figure
from bokeh.transform import cumsum, factor_cmap
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Legend, LegendItem, LabelSet, RangeTool, HoverTool, DatetimeTickFormatter, WheelZoomTool, WheelPanTool, UndoTool, RedoTool, ResetTool, SaveTool, Whisker, NumeralTickFormatter, PanTool

from bokeh.models import ColumnDataSource, RangeTool, HoverTool, DatetimeTickFormatter, WheelZoomTool, WheelPanTool, UndoTool, RedoTool, ResetTool, SaveTool

In [2]:
true = True
data = {"specialization":{"0":"spec2","1":"spec2","2":"spec3","3":"spec3","4":"spec3","5":"spec1","6":"spec3","7":"spec3","8":"spec3","9":"spec3","10":"spec3","11":"spec3"},"job_title":{"0":"TestVacancy","1":"TestVacancy","2":"TestVacancy","3":"TestVacancy","4":"TestVacancy","5":"TestVacancy","6":"TestVacancy","7":"TestVacancy","8":"TestVacancy","9":"TestVacancy","10":"TestVacancy","11":"TestVacancy"},"min_salary":{"0":10,"1":100,"2":1000,"3":1,"4":10000,"5":10000,"6":1000000,"7":10,"8":200000,"9":300000,"10":400000,"11":500000},"max_salary":{"0":20000000,"1":200000,"2":20000,"3":2000,"4":200000,"5":20000,"6":20000,"7":2000,"8":20000000,"9":200000000,"10":20000000,"11":20000000},"experience":{"0":"\u041e\u0442 1 \u0434\u043e 3","1":"\u041e\u0442 1 \u0434\u043e 3","2":"\u041e\u0442 1 \u0434\u043e 3","3":"\u041e\u0442 1 \u0434\u043e 3","4":"\u041e\u0442 1 \u0434\u043e 3","5":"\u041e\u0442 1 \u0434\u043e 3","6":"\u041e\u0442 1 \u0434\u043e 3","7":"\u041e\u0442 1 \u0434\u043e 3","8":"\u041e\u0442 2 \u0434\u043e 3","9":"\u041e\u0442 1 \u0434\u043e 3","10":"\u041e\u0442 1 \u0434\u043e 3","11":"\u041e\u0442 1 \u0434\u043e 3"},"key_skills":{"0":"[Python, Git]","1":"[Python, Git]","2":"[Python, Git]","3":"[Python, Git]","4":"[Python, Git]","5":"[Python, Git]","6":"[Python, Git]","7":"[Python, Git]","8":"[Python, Git]","9":"[Python, Git]","10":"[Python, Git]","11":"[Python, Git]"},"published_at":{"0":"2023-07-17T00:00:00","1":"2023-07-17T00:00:00","2":"2023-07-17T00:00:00","3":"2023-07-17T00:00:00","4":"2023-07-17T00:00:00","5":"2023-07-17T00:00:00","6":"2023-07-17T00:00:00","7":"2023-07-17T00:00:00","8":"2023-07-17T00:00:00","9":"2023-07-17T00:00:00","10":"2023-07-18T00:00:00","11":"2024-07-17T00:00:00"},"status":{"0":true,"1":true,"2":true,"3":true,"4":true,"5":true,"6":true,"7":true,"8":true,"9":true,"10":true,"11":true},"city":{"0":0,"1":0,"2":0,"3":0,"4":0,"5":"Moskow","6":"Novosibirsk","7":0,"8":0,"9":0,"10":0,"11":0},"schedule":{"0":0,"1":0,"2":0,"3":0,"4":0,"5":"Full-time","6":"part-time","7":"flex-time","8":0,"9":0,"10":0,"11":0}}
df = pd.DataFrame(data)
df.head()

,specialization,job_title,min_salary,max_salary,experience,key_skills,published_at,status,city,schedule
0,spec2,TestVacancy,10,20000000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
1,spec2,TestVacancy,100,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
2,spec3,TestVacancy,1000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
3,spec3,TestVacancy,1,2000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
4,spec3,TestVacancy,10000,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0


In [3]:
size = 10 #размер текста на графиках
missing = 0 #замена для пустых клеток
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
import bokeh
# bokeh.sampledata.download()

In [6]:
def counts(df_full, name):
    df = df_full.loc[df_full[name] != missing]
    df = df.groupby(name)[name].count().rename("counts").reset_index()
    
    return df


def avg_sal(df, df_bool = False):
    """Вспомогательная функция получения ср.зп"""
    if df_bool == True: #Режим возврата массива
        return ((df['min_salary'] + df['max_salary'])/2).rename("ср.зарплата")
    avg = int(((df['min_salary'] + df['max_salary'])/2).mean())
    return avg

In [14]:
def DateLine(df_full, height = 300):
    """График DateLine типа scatter для дат по кол. вакансий за день"""
    color = "orange"
    background_color = "#efefef"
    y_label = "Вакансий опубликованно за день"

    # Создание источника данных
    df = counts(df_full, "published_at")
    dates = np.array(pd.to_datetime(df["published_at"]), dtype=np.datetime64)
    source = ColumnDataSource(data=dict(date=dates, y = df["counts"]))
    
    # Создание графика
    fig = figure(height= int(height/3*2), background_fill_color=background_color, #визуальные настройки
                 tools = "", toolbar_location="right",x_axis_type="datetime", x_axis_location="below",x_range=(dates.min(), dates.max())) #настройки
    
    # Панель инструментов
    zoom = WheelZoomTool(dimensions="width", name = "Увеличение", description = "Увеличить по оси x")
    fig.add_tools(zoom)
    fig.add_tools(WheelPanTool(dimension = "width", name = "Передвижение", description = "Передвижение по оси x"))
    fig.add_tools(UndoTool(name = "Назад", description = "Отменить действие"))
    fig.add_tools(RedoTool(name = "Вперёд", description = "Вернуть действие"))
    fig.add_tools(ResetTool(name = "Сброс", description = "Сброс до начального положения"))
    fig.add_tools(SaveTool(name = "Сохранить", description = "Сохранить текущий график как картинку"))
    
    # Активные инструменты
    fig.toolbar.active_drag = None
    fig.toolbar.active_scroll = zoom
    
    #Hover
    hover = HoverTool(tooltips=[("Дата", "$x{%d/%m/%Y}"), ("Вакансий опубликованно за день", "$y{0a}")], formatters={"$x": "datetime"}, name = "Описание", description = "Показывать описание точек")
    fig.add_tools(hover)
    
    #Форматирование дат оси x
    fig.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M", days="%d/%m", months="%m/%y", years="%Y")
    
    #Элементы и визуализация графика
    fig.scatter('date', 'y', source=source, 
                size=10, color='black', line_color = color, line_width = 3, alpha=1, level = "overlay") #визуальные настройки
    fig.line('date', 'y', source=source,
            line_color = color, line_width = 5)#визуальные настройки
    fig.yaxis.axis_label = y_label
    fig.toolbar.autohide = True
    fig.background_fill_alpha = 0
    fig.border_fill_alpha = 0
    fig.grid.grid_line_color = "black"  # Цвет сетки - черный
    fig.grid.grid_line_alpha = 0.8  # Прозрачность сетки - яркая
    fig.outline_line_alpha = 1
    fig.outline_line_color = "black"

    #Создание ползунка
    select = figure(height = int(height / 3), y_range=fig.y_range,
                    x_axis_type="datetime", y_axis_type=None,
                    tools="", toolbar_location=None, background_fill_color=background_color)

    range_tool = RangeTool(x_range=fig.x_range)
    range_tool.overlay.fill_color = "green"
    range_tool.overlay.fill_alpha = 0.3
    
    #Настрока элементов ползунка и визуализации
    select.scatter('date', 'y', source=source, 
                size=10, color='black', line_color = color, line_width = 3, alpha=1, level = "overlay") #визуальные настройки
    select.line('date', 'y', source=source,
                line_color = color, line_width = 5)#визуальные настройки
    select.add_tools(range_tool)
    select.background_fill_alpha = 0
    select.border_fill_alpha = 0
    select.grid.grid_line_color = "black"  # Цвет сетки - черный
    select.grid.grid_line_alpha = 0.8  # Прозрачность сетки - яркая
    select.outline_line_alpha = 1
    select.outline_line_color = "black"
    
    # Настройки оси х
    select.xaxis.major_label_orientation = 1.2
    select.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M", days="%d/%m", months="%m/%y", years="%Y")
    
    
    show(column(fig, select))
    
DateLine(df)


In [17]:
from math import pi
import numpy as np
from bokeh.io import output_file, show
from bokeh.palettes import Category10
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import Legend, HoverTool, LegendItem, LabelSet, ColumnDataSource
def LevelPie(df_full, height = 300):
    """График Levelpie типа Pie по соотношению групп опыта"""
    df = counts(df_full, "experience")
    
    #Источник данных
    df = counts(df_full, "experience")
    
    # Задаем значения для диаграммы
    value = df["counts"].tolist()

    # Создаем словарь с данными
    data = {
        'name': df["experience"].tolist(),  # Метки для каждого сектора
        'value': value,  # Значения для каждого сектора
        'angle': [v/sum(value)*2*pi for v in value],  # Угол для каждого сектора
        'cumulative_angle':[(sum(value[0:i+1])- (item/2))/sum(value)*2*pi for i,item in enumerate(value)],  # Кумулятивный угол для каждого сектора
        'percentage': [d/sum(value)*100 for d in value],  # Процент для каждого сектора
        'color': colors[:len(value)]  # Цвет для каждого сектора
    }

    # Создаем метки в процентах
    data['label'] = ["{:.0f}%".format(p) for p in data['percentage']]

    # Вычисляем координаты для меток
    data['cos'] = np.cos(data['cumulative_angle'])*0.3
    data['sin'] = np.sin(data['cumulative_angle'])*0.3

    # Создаем источник данных для Bokeh
    source = ColumnDataSource(data=data)

    # Настройки для всплывающих подсказок
    TOOLTIPS = [
        ("Название", "@name"),  # Название сектора
        ("Кол.вакансий", "@value{0a}"),  # Значение сектора
        ("Процент", "@percentage{0.2f}%")  # Процент сектора
    ]

    # Создаем объект Figure для построения графика
    fig = figure(height = height,x_range=(-0.5, 0.5), y_range=(-1, 1), tools='hover', tooltips=TOOLTIPS, toolbar_location=None)
    
    # Создаем кольцевые секторы диаграммы
    r = fig.annular_wedge(x=0, y=0, inner_radius=0.2, outer_radius=0.4,
                        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                        fill_color='color', fill_alpha=1, direction = 'anticlock', line_alpha = 1, line_color = "black", line_width = 5,
                        source=source)

    # Создаем легенду
    legend = Legend(items=[LegendItem(label=dict(field="name"), renderers=[r])], location=(0, 80), label_text_font_size = str(size) + "px")
    legend.background_fill_alpha = 0
    legend.border_line_alpha = 0
    fig.add_layout(legend, 'right')

    # Добавляем метки
    labels = LabelSet(x='cos', y='sin', text="label", y_offset=0,
                      text_font_size = str(int(height / 40)) + "px", text_color="black",
                      source=source, text_align='center')
    fig.add_layout(labels)

    # Настраиваем внешний вид графика
    fig.axis.axis_label=None
    fig.axis.visible=False
    fig.grid.grid_line_color = None
    fig.outline_line_color = None
    fig.background_fill_alpha = 0
    fig.border_fill_alpha = 0
    
    #Настраиваем интерактивную легенду
    fig.legend.click_policy="hide"

    # Отображаем график
    show(fig)
LevelPie(df)

In [9]:
df.head()

,specialization,job_title,min_salary,max_salary,experience,key_skills,published_at,status,city,schedule
0,spec2,TestVacancy,10,20000000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
1,spec2,TestVacancy,100,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
2,spec3,TestVacancy,1000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
3,spec3,TestVacancy,1,2000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
4,spec3,TestVacancy,10000,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0


# Boxes


In [16]:
def SalaryBox(df_full, height = 300):
    """График SalaryBox типа vbox для отображения зарплат по пыту работы"""
    df = pd.concat([df_full.loc[:, "experience"], avg_sal(df_full.loc[:, ["min_salary", "max_salary"]], df_bool = True) ], axis=1)

    kinds = df["experience"].unique()

    # вычисляем квантили
    qs = df.groupby("experience")["ср.зарплата"].quantile([0.25, 0.5, 0.75])
    qs = qs.unstack().reset_index()
    qs.columns = ["experience", "q1", "q2", "q3"]
    df = pd.merge(df, qs, on="experience", how="left")

    # вычисляем границы выбросов по IQR
    iqr = df.q3 - df.q1
    df["upper"] = df.q3 + 1.5*iqr
    df["lower"] = df.q1 - 1.5*iqr

    source = ColumnDataSource(df)

    fig = figure(height = height,x_range=kinds, tools="", toolbar_location="right",
               background_fill_color="#eaefef", y_axis_label="Руб.")

    # границы выбросов
    whisker = Whisker(base="experience", upper="upper", lower="lower", source=source)
    whisker.upper_head.size = whisker.lower_head.size = 20
    fig.add_layout(whisker)

    # ящики с квантилями
    cmap = factor_cmap("experience", "TolRainbow7", kinds)
    fig.vbar("experience", 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
    fig.vbar("experience", 0.7, "q1", "q2", source=source, color=cmap, line_color="black")

    # Визуальная часть
    fig.xgrid.grid_line_color = None
    fig.axis.major_label_text_font_size="14px"
    fig.axis.axis_label_text_font_size="12px"
    fig.yaxis[0].formatter = NumeralTickFormatter(format="0")
    fig.outline_line_color = None
    fig.background_fill_alpha = 0
    fig.border_fill_alpha = 0
    
    #Hover
    hover = HoverTool(tooltips=[
        ("Группа", "@experience"),
        ("Минимум", "@lower{0a}"),
        ("Медиана", "@q2{0a}"),
        ("Максимум", "@upper{0a}")
    ], mode='vline', line_policy="interp", toggleable = True)
    
    # hover.formatters = {"@experience": "printf", "@lower": "printf", "@q2": "printf", "@upper": "printf"}
    # hover.formatters["@experience"] = "{'printf': function(x) { return x.substring(0, 4); }}"
    # hover.formatters["@lower"] = "{'printf': function(x) { return x.toFixed(1); }}"
    # hover.formatters["@q2"] = "{'printf': function(x) { return x.toFixed(1); }}"
    # hover.formatters["@upper"] = "{'printf': function(x) { return x.toFixed(1); }}"
    
    fig.add_tools(hover)
    
     # Панель инструментов
    zoom = WheelZoomTool(dimensions="height", name = "Увеличение", description = "Увеличить по оси x")
    pan =PanTool(dimensions = "height", name = "Передвижение", description = "Передвижение по оси x")
    fig.add_tools(zoom)
    fig.add_tools(pan)
    fig.add_tools(ResetTool(name = "Сброс", description = "Сброс до начального положения"))
    fig.add_tools(SaveTool(name = "Сохранить", description = "Сохранить текущий график как картинку"))
    
    # Активные инструменты
    fig.toolbar.active_drag = pan
    fig.toolbar.active_scroll = zoom
    
    # Настройки отображения чисел на оси y
    fig.yaxis[0].formatter = NumeralTickFormatter(format="0.0a")


    show(fig)
SalaryBox(df)

In [11]:
df

,specialization,job_title,min_salary,max_salary,experience,key_skills,published_at,status,city,schedule
0,spec2,TestVacancy,10,20000000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
1,spec2,TestVacancy,100,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
2,spec3,TestVacancy,1000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
3,spec3,TestVacancy,1,2000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
4,spec3,TestVacancy,10000,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
5,spec1,TestVacancy,10000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,Moskow,Full-time
6,spec3,TestVacancy,1000000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,Novosibirsk,part-time
7,spec3,TestVacancy,10,2000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,flex-time
8,spec3,TestVacancy,200000,20000000,От 2 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
9,spec3,TestVacancy,300000,200000000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0


In [26]:
def counts(df_full, name, salary = False):
    """Вспомогательная функция упорядочивания df по колл. вакансий в зависимости от столбца для группировки"""
    df = df_full.loc[df_full[name] != missing]
    if salary == True:
        df_sal = pd.concat([df.loc[:, name], avg_sal(df.loc[:, ["min_salary", "max_salary"]], df_bool = True) ], axis=1).groupby(name)["ср.зарплата"].sum()
        df_counts = df.groupby(name)[name].count().rename("counts")
        df = pd.concat([df_sal, df_counts], axis = 1).reset_index()
    else:
        df = df.groupby(name)[name].count().rename("counts").reset_index()
    
    return df


def to_table(df, head = []):
    thead = ""
    for i in head:
        thead += f"<th> {i}</th>"

    table = f"""
    <table>
    <thead>
    <tr>
    {thead}
    </tr>
    </thead>
    <tbody>
    """
    
    for index, row in df.iterrows():
        elements = ""
        for i in row.values:
            elements += f"<td>{i}</td>"
        table += f"<tr>{elements}</tr>"
   
    table += "</tbody></table>"
    
    return table
to_table(counts(df, "schedule",salary = True), ["Название", "Кол.вакансий", "Ср.зарплата"])

'\n    <table>\n    <thead>\n    <tr>\n    <th> Название</th><th> Кол.вакансий</th><th> Ср.зарплата</th>\n    </tr>\n    </thead>\n    <tbody>\n    <tr><td>Full-time</td><td>15000.0</td><td>1</td></tr><tr><td>flex-time</td><td>1005.0</td><td>1</td></tr><tr><td>part-time</td><td>510000.0</td><td>1</td></tr></tbody></table>'

In [13]:
df

,specialization,job_title,min_salary,max_salary,experience,key_skills,published_at,status,city,schedule
0,spec2,TestVacancy,10,20000000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
1,spec2,TestVacancy,100,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
2,spec3,TestVacancy,1000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
3,spec3,TestVacancy,1,2000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
4,spec3,TestVacancy,10000,200000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
5,spec1,TestVacancy,10000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,Moskow,Full-time
6,spec3,TestVacancy,1000000,20000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,Novosibirsk,part-time
7,spec3,TestVacancy,10,2000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,flex-time
8,spec3,TestVacancy,200000,20000000,От 2 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
9,spec3,TestVacancy,300000,200000000,От 1 до 3,"[Python, Git]",2023-07-17T00:00:00,True,0,0
